In [1]:
import os
import numpy as np
from pathlib import Path
import mne
from mne import create_info
from mne.io import RawArray
from mne_bids import BIDSPath, get_entity_vals, read_raw_bids, get_entities_from_fname
from eztrack.fragility import DiscreteLinearSystem
from eztrack.fragility.fragility import _reference_raw, state_lds_derivative
from eztrack.io.array import DerivativeArray
from eztrack.io.deriv_info import create_deriv_info

In [2]:
bids_root = "D:/OneDriveParent/OneDrive - Johns Hopkins/scalp_reconstruction/ICA"
sourcedata = Path(bids_root) / "sourcedata"
deriv_dir = Path(bids_root) / "derivatives"
figure_dir = Path(deriv_dir) / "figures"

In [3]:
session = 'monitor'
task = 'monitor'
run = "01"
datatype = "eeg"
montage = "standard_1020"
ext = ".edf"
reference="monopolar"

In [4]:
subjects = get_entity_vals(bids_root, 'subject')
bids_fpaths = [BIDSPath(subject=s, session=session, task=task, run=run, datatype=datatype, extension=ext, root=bids_root).match() for s in subjects]
bids_fpaths = [item for sublist in bids_fpaths for item in sublist]

In [5]:
reconstructed_root = Path(deriv_dir) / "reconstructed"
reconstructed_fpaths = [BIDSPath(subject=s, session=session, task=task, run=run, datatype=datatype, extension="fif", root=reconstructed_root).match() for s in subjects]
reconstructed_fpaths = [item for sublist in reconstructed_fpaths for item in sublist]

In [6]:
fpath_pairs = [(b, r) for b,r in zip(bids_fpaths, reconstructed_fpaths)]

In [7]:
for raw_fpath, reconstructed_fpath in fpath_pairs:
    raw_bids_entities = get_entities_from_fname(raw_fpath)
    reconstructed_bids_entities = get_entities_from_fname(reconstructed_fpath)
    raw = read_raw_bids(raw_fpath)
    reconstructed = read_raw_bids(reconstructed_fpath)
    raw_fig = raw.plot(show=False)
    reconstructed_fig = reconstructed.plot(show=False)
    raw_root = figure_dir / "raw" 
    raw_fig_bids = BIDSPath(**raw_bids_entities, root=raw_root)
    raw_figs_fpath = str(raw_fig_bids).split(".")[0] + ".png"
    raw_fig.savefig(raw_figs_fpath)
    reconstructed_root = figure_dir / "reconstructed"
    reconstructed_fig_bids = BIDSPath(**reconstructed_bids_entities, root=reconstructed_root)
    reconstructed_figs_fpath = str(reconstructed_fig_bids).split(".")[0] + ".png"
    reconstructed_fig.savefig(reconstructed_figs_fpath)